In [2]:
dataset_id = "imdb_modeled"
!bq --location=US mk --dataset {dataset_id} 

BigQuery error in mk operation: Dataset 'starry-center-266501:imdb_modeled'
already exists.


## Create new table "People" from columns of imdb_staging.NameBasics. Establish nConst as a primary key. Retitle "string_field_i" as correct name. Cast birthYear and deathYear from string to int

%%bigquery
create table imdb_modeled.People as 
SELECT distinct string_field_0 as nConst, string_field_1 as primaryName, safe_cast(string_field_2 as int64) as birthYear,
safe_cast(string_field_3 as int64) as deathYear, string_field_4 as primaryProfession, string_field_5 as knownForTitles
FROM imdb_staging.NameBasics

### Sample People

In [16]:
%%bigquery 
SELECT * from imdb_modeled.People limit 5

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm1072898,Hans-Joachim Frank,1954,NaN,"actor,director,assistant_director","tt1091772,tt0078264,tt0302888,tt1191976"
1,nm0074890,Bärbl Bergmann,1931,2003.0,"director,writer,assistant_director","tt0130988,tt0310958,tt1298811,tt0085199"
2,nm2228559,Juan Souter,1970,NaN,"art_department,set_decorator","tt2980592,tt1263670,tt0406759,tt0806203"
3,nm0392646,Jes Holtsø,1956,NaN,"actor,art_department,composer","tt0075002,tt0073482,tt0066807,tt0068081"
4,nm0884620,Frantisek Valert,1932,1993.0,"cinematographer,camera_department,special_effects","tt0058971,tt0062332,tt0232750,tt0405822"


### Verify nConst as a primary key

In [15]:
%%bigquery
SELECT COUNT(*) FROM imdb_modeled.People

,f0_
0,9870691


In [22]:
%%bigquery
SELECT COUNT(DISTINCT nConst) FROM imdb_modeled.People

,f0_
0,9870691


## Create table "Titles" from columns of imdb_staging.TitleBasics. Establish primary key tConst. Cast strings endYear and runtimeMinutes to integers

In [2]:
%%bigquery
CREATE TABLE imdb_modeled.Titles as 
SELECT DISTINCT tConst, titleType, primaryTitle, startYear, safe_cast(endYear as int64) as endYear,  
safe_cast(runtimeMinutes as int64) as runtimeMinutes, genres
FROM imdb_staging.TitleBasics

""


### Sample Titles

In [28]:
%%bigquery
SELECT * from imdb_modeled.Titles 
LIMIT 5

,tConst,titleType,primaryTitle,startYear,endYear,runtimeMinutes
0,tt10894890,tvEpisode,Episode dated 2 March 1999,1999,None,NaN
1,tt0607122,tvEpisode,Episode dated 29 November 1999,1999,None,NaN
2,tt0488234,short,Tucumán arde: documento,1999,None,NaN
3,tt2096773,tvEpisode,Episode #1.76,1999,None,NaN
4,tt10710646,tvEpisode,Taose! Kuraken,1999,None,20.0


### Verify tConst as a primary key 

In [29]:
%%bigquery
SELECT COUNT(*) from imdb_modeled.Titles

,f0_
0,6524093


In [33]:
%%bigquery
SELECT COUNT (DISTINCT tConst)from imdb_modeled.Titles

,f0_
0,6524093


## Create table "Writes" from writers and tConst columns of imdb_staging.TitleCrew. Establish primary key tConst. Use parent table Titles to place foreign key restriction on Writes for the tConst field. Name unnamed fields of TitleCrew.

In [15]:
%%bigquery
create table imdb_modeled.Writes as 
SELECT DISTINCT tc.string_field_2 as writers, tc.string_field_0 as tConst
FROM imdb_staging.TitleCrew tc
INNER JOIN imdb_modeled.Titles ti on tc.string_field_0 = ti.tConst

""


### Verify tConst as a primary key of Writes

In [16]:
%%bigquery
SELECT COUNT (*) 
FROM imdb_modeled.Writes

,f0_
0,6524093


In [17]:
%%bigquery
SELECT COUNT (DISTINCT tConst)
FROM imdb_modeled.Writes

,f0_
0,6524093


### Verify imdb_modeled.Writes has a foreign key constraint on field tConst by parent imdb_modeled.Titles

In [18]:
%%bigquery
SELECT wi.tConst
FROM imdb_modeled.Writes wi
WHERE wi.tConst NOT IN (SELECT ti.tConst FROM imdb_modeled.Titles ti)

,tConst


## Create table "Directs" from directors and tConst columns of imdb_staging.TitleCrew. Create primary key of tConst. Place foreign key restriction on Directors for the tConst field of parent table imdb_modeled.Titles. Name unnamed fields of TitleCrew.

In [19]:
%%bigquery
CREATE table imdb_modeled.Directs as 
SELECT DISTINCT tc.string_field_1 as directors, tc.string_field_0 as tConst
FROM imdb_staging.TitleCrew tc
INNER JOIN imdb_modeled.Titles ti on tc.string_field_0 = ti.tConst

""


### Sample Directs

In [20]:
%%bigquery
SELECT * 
FROM imdb_modeled.Directs
LIMIT 5

,directors,tConst
0,nm0299154,tt0011983
1,nm0510357,tt0036917
2,nm0742819,tt0082495
3,nm0815638,tt0286200
4,nm1143942,tt0326989


## Verify tConst as a primary key 

In [21]:
%%bigquery
SELECT COUNT(*) FROM imdb_modeled.Directs

,f0_
0,6524093


In [22]:
%%bigquery
SELECT COUNT(DISTINCT tConst) FROM imdb_modeled.Directs

,f0_
0,6524093


### Verify imdb_modeled.Direct has a foreign key constraint on field tConst by parent imdb_modeled.Titles

In [23]:
%%bigquery
SELECT di.tConst
FROM imdb_modeled.Directs di
WHERE di.tConst NOT IN (SELECT ti.tConst FROM imdb_modeled.Titles ti)

,tConst


## Create table "TitleRatings" from imdb_modeled.TitleRatings, with primary key tConst and foreign key constraint on tConst by parent table Titles

In [54]:
%%bigquery
CREATE table imdb_modeled.TitleRatings as
SELECT distinct tr.tConst, averageRating, numVotes
FROM imdb_staging.TitleRatings tr
INNER JOIN imdb_staging.TitleBasics tb on tb.tConst = tr.tConst

""


## Sample TitleRatings

In [55]:
%%bigquery
SELECT * 
FROM imdb_modeled.TitleRatings 
LIMIT 5

,tConst,averageRating,numVotes
0,tt0049468,4.1,319
1,tt0077533,4.2,5864
2,tt0050514,4.1,38
3,tt0077620,4.2,212
4,tt0064307,3.2,607


### Verify tConst as a primary key

In [56]:
%%bigquery
SELECT COUNT(*) 
FROM imdb_modeled.TitleRatings

,f0_
0,1022151


In [57]:
%%bigquery
SELECT COUNT(DISTINCT tConst)
FROM imdb_modeled.TitleRatings

,f0_
0,1022151


### Verify imdb_modeled.TitleRatings has a foreign key constraint on field tConst by parent imdb_modeled.Titles

In [2]:
%%bigquery
SELECT tr.tConst
FROM imdb_modeled.TitleRatings tr
WHERE tr.tConst NOT IN (SELECT ti.tConst FROM imdb_modeled.Titles ti)

,tConst


## Create table Characters with fields from imdb_staging.TitlePrincipals with primary key tConst, and foreign key restraint on field nConst by parent table People

In [58]:
%%bigquery
CREATE table imdb_modeled.Characters as
SELECT DISTINCT tConst, tp.nConst, job, characters
FROM imdb_staging.TitlePrincipals tp
INNER JOIN imdb_modeled.People pe 
ON pe.nConst = tp.nConst

""


## Sample Characters

In [59]:
%%bigquery 
SELECT *
FROM imdb_modeled.Characters
LIMIT 5

,tConst,nConst,job,characters
0,tt4533768,nm0233893,\N,"[""Tilly Winn""]"
1,tt7328956,nm2362231,adaptation,\N
2,tt4844918,nm6056794,\N,\N
3,tt10514508,nm10779526,\N,\N
4,tt10987232,nm0616338,screenplay,\N


### Verify tConst as a primary key

In [60]:
%%bigquery
SELECT COUNT (*)
FROM imdb_modeled.Characters

,f0_
0,37687123


In [61]:
%%bigquery
SELECT COUNT (DISTINCT tConst) FROM imdb_modeled.Characters

,f0_
0,5899527


### Verify foreign key constraint on nConst by parent table People 

In [ ]:
%%bigquery
SELECT ch.nConst
FROM imdb_modeled.Characters ch
WHERE ch.nConst NOT IN (SELECT pe.nConst FROM imdb_modeled.People pe)

In [1]:
# Using Apache Beam, create a table Directs_Beam, that makes Directs 3NF Compliant 
# Changes nConst field from an array of directors to a single primary director 
%run Directs_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'imdb_modeled'
 projectId: 'starry-center-266501'
 tableId: 'Directs'> referenced by query SELECT directors, tConst FROM imdb_modeled.Directs limit 100
INFO:apache_beam.io.gcp.bigquery_tools:Created table starry-center-266501.imdb_modeled.Directs_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 f

# Verify that Directs_Beam has a primary key tConst

In [1]:
%%bigquery
select count(*) from imdb_modeled.Directs_Beam

,f0_
0,100


In [9]:
%%bigquery
select count(distinct tConst) from imdb_modeled.Directs_Beam

,f0_
0,100


# Verify that Directs_Beam has a foreign key constraint placed on it by Titles

In [10]:
%%bigquery 
SELECT db.tConst
FROM imdb_modeled.Directs_Beam db
WHERE db.tConst NOT IN (SELECT t.tConst FROM imdb_modeled.Titles t)

,tConst


In [ ]:
# Using Apache Beam, create a table Directs_Beam, that has scalar values in the tConst field
%run directs_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


## Sample directs_beam_dataflow

In [1]:
%%bigquery
select * from imdb_modeled.Directs_Beam_DF limit 5

,director,tConst
0,nm1949089,tt11683962
1,nm0559400,tt8927858
2,nm1191303,tt6671538
3,nm0637176,tt11128110
4,nm0902823,tt6647040


## Confirm Directs_beam_dataflow has composite primary key nConst, tConst, and that it is a child table to parent table People

In [2]:
%%bigquery 
SELECT COUNT (*) FROM imdb_modeled.Directs_Beam_DF

,f0_
0,3111973


In [3]:
%%bigquery
SELECT COUNT(*) FROM 
(SELECT DISTINCT director, tConst 
FROM imdb_modeled.Directs_Beam_DF)
# verified composite primary key

,f0_
0,3111973


In [4]:
%%bigquery
# verifying FK restraint placed on nConst by people
SELECT director
FROM imdb_modeled.Directs_Beam_DF
WHERE director NOT IN (SELECT nConst FROM imdb_modeled.People) 

,director


In [6]:
%%bigquery 
#verifying FK restraint placed on tConst by Titles
SELECT dbd.tConst FROM imdb_modeled.Directs_Beam_DF dbd
WHERE dbd.tConst NOT IN (SELECT t.tConst FROM imdb_modeled.Titles t)

,tConst


In [1]:
# Writes_beam
%run Writes_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.internal.gcp.auth:Setting socket default timeout to 60 seconds.
INFO:apache_beam.internal.gcp.auth:socket default timeout is 60.0 seconds.
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'imdb_modeled'
 projectId: 'starry-center-266501'
 tableId: 'Writes'> referenced by query SELECT writers, tConst FROM imdb_modeled.Writes limit 100
INFO:apache_beam.io.gcp.bigquery_tools:Created table starry-center-266501.imdb_modeled.Writes_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fields

In [5]:
# Writes_beam
%run Writes_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


### Sample Writes_beam_DF, prove writer, tConst as a composite primary key, and prove it is a child table to parent tables People and Titles

In [1]:
%%bigquery
SELECT * FROM imdb_modeled.Writes_Beam_DF LIMIT 5

,Writer,tConst
0,nm10641662,tt11110374
1,nm5597421,tt2800438
2,nm5366481,tt6870672
3,nm5168876,tt5794264
4,nm8278497,tt7110898


In [2]:
%%bigquery
SELECT COUNT (*) FROM imdb_modeled.Writes_Beam_DF 

,f0_
0,1623309


In [3]:
%%bigquery 
SELECT COUNT (*) FROM
(SELECT DISTINCT writer, tConst
FROM imdb_modeled.Writes_Beam_DF)
# verified composite primary key

,f0_
0,1623309


In [4]:
%%bigquery 
# verfiying FK restraint placed on nConst by People
SELECT writer FROM imdb_modeled.Writes_Beam_DF wbd
WHERE wbd.writer NOT IN (SELECT p.nConst FROM imdb_modeled.People p) 

,writer


In [5]:
%%bigquery 
# verifying FK restraint placed on tConst by Titles
SELECT wbd.tConst FROM imdb_modeled.Writes_Beam_DF wbd
WHERE wbd.tConst NOT IN (SELECT t.tConst FROM imdb_modeled.Titles t)

,tConst


In [2]:
# Writes_beam
%run KnownForTitles_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'imdb_modeled'
 projectId: 'starry-center-266501'
 tableId: 'People'> referenced by query SELECT nConst, knownForTitles FROM imdb_modeled.People limit 100
INFO:apache_beam.io.gcp.bigquery_tools:Created table starry-center-266501.imdb_modeled.knownForTitles_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'nConst'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'tConst'
 type: 'STRING'>]>. Result: <Table
 creationTime: 1583549015615
 etag: 'geXp9xpAxFiiisf1FE32pQ=='


In [ ]:
# Separate off the knownForTitles arrays from people to make scalar in a different table
%run knownForTitles_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


## Sample knownForTitles_Beam_DF, show that it has a composite primary key of nConst and tConst, and show that it is a child table of parent table People

In [7]:
%%bigquery
SELECT * FROM imdb_modeled.knownForTitles_Beam_DF LIMIT 5

,nConst,tConst
0,nm10100311,\N
1,nm10346006,tt9296836
2,nm1102887,tt0134453
3,nm0599977,tt0150508
4,nm2359980,tt0847192


In [8]:
%%bigquery
SELECT COUNT (*) FROM imdb_modeled.knownForTitles_Beam_DF 

,f0_
0,17254141


In [10]:
%%bigquery 
# verfiying composite PK nConst, tConst
SELECT COUNT (*) FROM 
(SELECT DISTINCT nConst, tConst FROM imdb_modeled.knownForTitles_Beam_DF)

,f0_
0,17254141


In [12]:
%%bigquery 
# verfiying FK restraint placed by people
SELECT kbd.nConst FROM imdb_modeled.knownForTitles_Beam_DF kbd
WHERE kbd.nConst NOT IN (SELECT p.nConst FROM imdb_modeled.People p)

,nConst


In [12]:
%run primaryProfession_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'imdb_modeled'
 projectId: 'starry-center-266501'
 tableId: 'People'> referenced by query SELECT nConst, primaryProfession FROM imdb_modeled.People limit 100
INFO:apache_beam.io.gcp.bigquery_tools:Created table starry-center-266501.imdb_modeled.primaryProfession_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'nConst'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'primaryProfession'
 type: 'STRING'>]>. Result: <Table
 creationTime: 1583631059545
 etag: 'f47C6QsCG

In [14]:
%run primaryProfession_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


#### Sample primaryProfession_Beam_Dataflow, show that nConst and job make up a compsite primary key, and that it is a child table to parent table People

In [27]:
%%bigquery
SELECT * FROM imdb_modeled.primaryProfession_Beam_DF LIMIT 5

,nConst,primaryProfession
0,nm7987813,executive
1,nm0855685,casting_department
2,nm11071615,set_decorator
3,nm3917438,publicist
4,nm3083662,casting_director


In [17]:
%%bigquery
SELECT COUNT (*) FROM imdb_modeled.primaryProfession_Beam_DF

,f0_
0,10855861


In [22]:
%%bigquery
# verify primaryProfession_Beam_DF has a composite primary key of nConst and primaryProfession
SELECT COUNT (*) FROM (SELECT DISTINCT nConst, primaryProfession FROM imdb_modeled.primaryProfession_Beam_DF)

,f0_
0,10855861


In [26]:
%%bigquery 
# verify that primaryProfession has a foreign key constraint on nConst field by People
SELECT pr.nConst FROM imdb_modeled.primaryProfession_Beam_DF pr
WHERE pr.nConst NOT IN (SELECT p.nConst FROM imdb_modeled.People p)

,nConst


In [ ]:
# Replace imdb_modeled.People with imdb_modeled.Cast, such that Cast is standardized (doesn't include array attributes)

In [ ]:
%%bigquery
CREATE TABLE imdb_modeled.Cast as 
SELECT nConst, primaryName, birthYear, deathYear
FROM imdb_modeled.People

## Characters tables

In [3]:
# Characters_beam
%run Characters_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:oauth2client.transport:Refreshing due to a 401 (attempt 1/2)
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'imdb_modeled'
 projectId: 'starry-center-266501'
 tableId: 'Characters'> referenced by query SELECT tConst, nConst, job, characters FROM imdb_modeled.Characters limit 100


else: {'tConst': 'tt11043192', 'nConst': 'nm4290562', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt2644920', 'nConst': 'nm2005412', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt1182480', 'nConst': 'nm0942825', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt5627722', 'nConst': 'nm5807544', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt0248375', 'nConst': 'nm0180679', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt0171099', 'nConst': 'nm0646878', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt11023672', 'nConst': 'nm11017487', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt11086974', 'nConst': 'nm11040990', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt10209594', 'nConst': 'nm10633350', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt5566892', 'nConst': 'nm0155117', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt0417697', 'nConst': 'nm1665929', 'characters': '\\N', 'job': '\\N'}
else: {'tConst': 'tt10570

INFO:apache_beam.io.gcp.bigquery_tools:Created table starry-center-266501.imdb_modeled.Characters_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'tConst'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'nConst'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'job'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'characters'
 type: 'STRING'>]>. Result: <Table
 creationTime: 1583807275469
 etag: 'cH9dGO9PsRx7TtFBNgcCeQ=='
 id: 'starry-center-266501:imdb_modeled.Characters_Beam'
 kind: 'bigquery#table'
 lastModifiedTime: 1583807275507
 location: 'US'
 numBytes: 0
 numLongTermBytes: 0
 numRows: 0
 schema: <TableSchema
 fields: [<TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'tConst'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'nConst'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'job'
 type: 'STRING'>, <T

In [ ]:
# Characters_beam_dataflow
%run Characters_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))
INFO:apache_beam.runners.dataflow.internal.apiclient:Starting GCS upload to gs://imdb-beam/staging/split-characters-df.1583809008.385181/pipeline.pb...
INFO:apache_beam.runners.dataflow.internal.apiclient:Completed GCS upload to gs://imdb-beam/staging/split-characters-df.1583809008.385181/pipeline.pb in 0 seconds.
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/home/jupyter/venv/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmpxtezw0ph', 'apache-beam==2.19.0', '--no-deps', '--no-binary', ':all:']
INFO:apache_beam.runners.portability.stager:Staging SDK sources from PyPI to gs://imdb-beam/staging/split-characters-df.1583809008.385181/dataflow_pyth

#### Sample Characters_Beam_Dataflow, show that nConst, tConst and characters make up a compsite primary key, and that it is a child table to parent table People

In [4]:
%%bigquery
SELECT * FROM imdb_modeled.Characters_Beam_DF LIMIT 5

,tConst,nConst,job,characters
0,tt6646310,nm7443884,\N,Herself - Host
1,tt7744626,nm7672265,\N,Herself - Host
2,tt9620248,nm0407833,\N,Herself - Host
3,tt1723459,nm1216095,\N,Herself - Host
4,tt4858570,nm1548383,\N,Herself - Host


In [5]:
%%bigquery
SELECT COUNT (*) FROM imdb_modeled.Characters_Beam_DF

,f0_
0,37687123


In [14]:
%%bigquery
CREATE TABLE imdb_modeled.characters as
SELECT DISTINCT nConst, tConst, characters
FROM imdb_modeled.Characters_Beam_DF

""


In [1]:
%%bigquery
SELECT COUNT (*) FROM imdb_modeled.characters

,f0_
0,37687119


In [6]:
%%bigquery
# verify CHARACTERS_Beam_DF has a composite primary key of nConst and Characters
SELECT COUNT (*) FROM (SELECT DISTINCT nConst, tConst, characters FROM imdb_modeled.Characters_Beam_DF)

,f0_
0,37687119


In [7]:
%%bigquery 
# verify that primaryProfession has a foreign key constraint on nConst field by People
SELECT c.nConst FROM imdb_modeled.Characters_Beam_DF c
WHERE c.nConst NOT IN (SELECT p.nConst FROM imdb_modeled.People p)

,nConst
